In [1]:
#importing numpy and pandas libraries for data operations and data loading
import numpy as np 
import pandas as pd
#importing nltk and tensorflow preprosessing libraries for data processing and feature engineering
from time import time
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
import tensorflow as tf
#importing keras and tensorflow libraries for word embeddings and model building
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D, TimeDistributed,Flatten, GlobalMaxPool1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import multiply

import seaborn as sns
import matplotlib.pyplot as plt
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

#importing libraries for splitting the data and for tracking the progress
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [2]:
#Reading the training data from train.csv
train_df=pd.read_csv('train.csv')
#Reading the test data from test.csv
test_df=pd.read_csv('test.csv')

/Library/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#drop unnecessary columns like id,qid1,qid2 from train data
train_df = train_df.drop(['id','qid1','qid2'],axis=1)
#drop unnecessary columns like test_id from test data
test_df = test_df.drop(['test_id'],axis=1)
train_df.head()

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
train_df

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...
404285,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,What is one coin?,What's this coin?,0
404288,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [5]:
#Considering full amount of is_duplicate=1 data as it is small than is_duplicate=0
is_duplicate = train_df[train_df['is_duplicate']==1]
#considering 149260 rows of is_duplicate =0 data to maintain the balance of train data
not_duplicate = train_df[train_df['is_duplicate']==0].sample(149260)


In [6]:
#concating the is_duplicate and not_duplicate to form training data 
train_df = pd.concat([is_duplicate,not_duplicate])
train_df = train_df.sample(frac=1).reset_index(drop=True)
train_df

,question1,question2,is_duplicate
0,When did you start to question existence? I've...,Anyone who knows when Sons of guns starts agai...,0
1,What exactly is West Bank?,What is the West Bank?,1
2,I'm 20 years old and I wanted to know how can ...,I'm 19 years old and I wanted to know how can ...,1
3,What is a vibration?,What is pitch of vibrations?,0
4,What is the best way to study for history?,How do I study history?,1
...,...,...,...
298518,Is this good to move from core web development...,What turned your awesome job into hell?,0
298519,Does people of Sri Lanka burn the effigy of Rama?,Do people in Sri Lanka eat beef?,0
298520,"If I make homemade salad dressing, do I need t...",Can I store salad in refrigerator for a week?,0
298521,What is the correct way of using prepositions?,What's the right way to use prepositions?,1


In [7]:
# Defining a POS function to categorize the text to a particular parts of speech
def pos(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('R'):
        return wordnet.ADV
    elif tag.startswith('J'):
        return wordnet.ADJ
    else:
        return wordnet.NOUN

lemmatizer = WordNetLemmatizer()#Used Wordnet Lemmatizer to group different inflicted forms of a word together.

In [8]:
# Stacking up all the stopwords in english language 
STOPWORDS = set(stopwords.words('english'))

# Defining a clean text method
def clean_text(text):
    text = text.lower() #Converting the text to lower case.
    text = re.compile(r'<[^>]+>').sub(' ', text) #Replacing HTML tags with blank space
    text = re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+').sub('', text)#URLS
    text = re.compile(r'#(\w+)').sub('', text)#Replacing Hash tags  with blank space
    text = re.compile(r'@(\w+)').sub('', text)#Replacing @ tags with blank space
    text = re.compile(r"[^a-z0-9\-]").sub(' ', text)#Replacing unwanted words with blank space 
    
    
    
    for word in text.split():
        if word not in STOPWORDS and len(word)>3:
            text = " ".join(lemmatizer.lemmatize(word.strip(), pos(pos_tag([word.strip()])[0][1])))
            #filters and return the text after removing stopwords, lemmatization and pos tags
    
    return text

In [9]:
from gensim.models import KeyedVectors

In [10]:
E='GoogleNews-vectors-negative300.bin'


In [11]:

vocabulary = dict()
inverse_vocabulary = ['<unk>']  
#loading Google vectors for word2vec
word2vec = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
cols = ['question1', 'question2']
# Iterate through the questions of train and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():
        # Iterate through the text of both questions of the row
        for question in cols:
            q_2 = []
            # Iterate through each word in the question
            for word in clean_text(row[question]):
                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q_2.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q_2.append(vocabulary[word])
            # Replace questions with lists of word indices
            dataset.at[index,question]=q_2
            
dimensions = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, dimensions)  
embeddings[0] = 0  



In [12]:
# Create embedding matrix and assign each word to its word2vec embedding
for word, index in vocabulary.items():
    if word in list(word2vec.key_to_index):
        embeddings[index] = word2vec.word_vec(word)

del word2vec

/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
import itertools

In [14]:
# find the max length of a question from test and training datasets
x1=train_df.question1.map(lambda x: len(x)).max()
x2=train_df.question2.map(lambda x: len(x)).max()
x3=test_df.question1.map(lambda x: len(x)).max()
x4=test_df.question2.map(lambda x: len(x)).max()
length = max(x1,x2,x3,x4)
print(length)
# Split to train validation
validation = 40000
training_size = len(train_df) - validation
X = train_df[cols]
print(X.shape)
# Split the training and testing data into left and right inputs with left assigned to question1 and right to question2
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation)
X_train = {'left': X_train.question1, 'right': X_train.question2}
print(X_train['left'].shape)
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
print(len(X_validation['left']))
X_test = {'left': test_df.question1, 'right': test_df.question2}
# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values
# Padding the questions with zeroes to balance the input data
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=length)
print(X_train['left'].shape)

590
(298523, 2)
(258523,)
40000
(258523, 590)


In [15]:
# Defining hyperparameters for the model
hidden = 50
gradient_clipping_norm = 1.25
batch_size = 3000
epoch = 4
# Defining the MaLSTM Similarity function
def manhattan_distance(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))
# Defining left and right input
left = Input(shape=(length,), dtype='int32')
right = Input(shape=(length,), dtype='int32')
print(left,right)
#Defining the embedding layer
embedding_layer = Embedding(len(embeddings), dimensions, weights=[embeddings], input_length=length, trainable=False)
encoded_left = embedding_layer(left)
encoded_right = embedding_layer(right)
print(encoded_left)
# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(hidden)
left_o = shared_lstm(encoded_left)
right_o = shared_lstm(encoded_right)
# Calculates the distance as defined by the MaLSTM model
distance = Lambda(function=lambda x: manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_o, right_o])
# Combine everything to form the model
malstm = Model([left, right], [distance])
# Using Adadelta optimizer with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)
malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
start_time = time()
validation_data=([X_validation['left'], X_validation['right']], Y_validation)
print(len(validation_data[1]))
print(X_train['left'].shape)
# Training the MaLSTM model
malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=epoch,
                                validation_data=([X_validation['left'], X_validation['right']], Y_validation))
#print("Training time finished.\n{} epochs in {}".format(epoch, datetime.timedelta(seconds=time()-start_time)))

Tensor("input_1:0", shape=(None, 590), dtype=int32) Tensor("input_2:0", shape=(None, 590), dtype=int32)
Tensor("embedding_1/embedding_lookup/Identity_2:0", shape=(None, 590, 300), dtype=float32)


2022-04-17 17:02:34.017690: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-17 17:02:34.041256: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


40000
(258523, 590)


/Library/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 258523 samples, validate on 40000 samples
Epoch 1/4
258523/258523 [==============================] - 11781s 46ms/step - loss: 0.2481 - accuracy: 0.6493 - val_loss: 0.2309 - val_accuracy: 0.6573
Epoch 2/4
102000/258523 [==========>...................] - ETA: 1:53:48 - loss: 0.2267 - accuracy: 0.6576

KeyboardInterrupt: 